In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/lucy/processed/metadata')
db_path=os.path.join(root_dir,'db/lucy.db')

In [3]:
#define input tables and metadata
tbl_names_lst=["b_sample","l_sample"]

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    'LESS_GRP_VARS': False, # enforce bias in random queries toward smaller number of groupby vars. Default is no bias (i.e. uniform sampling)
    'LESS_CMP_VARS':False, # enforce bias in random queries toward small number of  comparison terms. Default is no bias (i.e. uniform sampling)
    'JOIN_CNDTN':{'WHERE':0.5, 'AND':0.5} #Use WHERE or AND with JOIN CLAUSE
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

# Generate Random  queries 

In [5]:
rnd_query=self.make_single_agg_query(agg_fntn=True)


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

no_of_rows=49812
no_of_cols=10
product=498.12


SELECT b_sample.HOSPBRTH, b_sample.PL_UIC20, b_sample.MEDINCST, b_sample.PL_RUCA4, l_sample.DX1, b_sample.NEOMAT, b_sample.DIED, b_sample.PL_UR_CA, COUNT(*), MIN(b_sample.AGE)  FROM b_sample  JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  GROUP BY b_sample.HOSPBRTH, b_sample.PL_UIC20, b_sample.MEDINCST, b_sample.PL_RUCA4, l_sample.DX1, b_sample.NEOMAT, b_sample.DIED, b_sample.PL_UR_CA


In [6]:
syn_tbl_name_lst=['b_sample_syn_01','l_sample_syn_01']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst,agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

REAL:
no_of_rows=772855
no_of_cols=11
product=8501.405


SYN:
no_of_rows=878922
no_of_cols=11
product=9668.142


SELECT b_sample.FEMALE, b_sample.PL_RUCA4, b_sample.PL_CBSA, b_sample.HISPANIC, l_sample.DX1, b_sample.NEOMAT, b_sample.PNUM_R, b_sample.RACE, b_sample.Homeless, COUNT(*), AVG(l_sample.DRGVER)  FROM b_sample  JOIN l_sample  ON b_sample.PNUM_R=l_sample.PNUM_R  GROUP BY b_sample.FEMALE, b_sample.PL_RUCA4, b_sample.PL_CBSA, b_sample.HISPANIC, l_sample.DX1, b_sample.NEOMAT, b_sample.PNUM_R, b_sample.RACE, b_sample.Homeless


SELECT b_sample_syn_01.FEMALE, b_sample_syn_01.PL_RUCA4, b_sample_syn_01.PL_CBSA, b_sample_syn_01.HISPANIC, l_sample_syn_01.DX1, b_sample_syn_01.NEOMAT, b_sample_syn_01.PNUM_R, b_sample_syn_01.RACE, b_sample_syn_01.Homeless, COUNT(*), AVG(l_sample_syn_01.DRGVER)  FROM b_sample_syn_01  JOIN l_sample_syn_01  ON b_sample_syn_01.PNUM_R=l_sample_syn_01.PNUM_R  GROUP BY b_sample_syn_01.FEMALE, b_sample_syn_01.PL_RUCA4, b_sample_syn_01.PL_CBSA, b_sample_syn_01.HISP